In [1]:
import os

In [2]:
%pwd

'/workspaces/End-to-End-MLOps-Pipeline/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-MLOps-Pipeline'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AshwinBalaji007/End-to-End-MLOps-Pipeline.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AshwinBalaji007"
os.environ["MLFLOW_TRACKING_PASSWORD"]="348ff2722ac77494369eb846bb75c50b7b698450"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from End_to_End_MLOps_Pipeline.constants import *
from End_to_End_MLOps_Pipeline.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/AshwinBalaji007/End-to-End-MLOps-Pipeline.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
import pandas as pd
import numpy as np
import joblib
import mlflow
import mlflow.sklearn
from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from End_to_End_MLOps_Pipeline.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log hyperparameters
            mlflow.log_params(self.config.all_params)

            # Log metrics
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Log metrics.json as artifact
            mlflow.log_artifact(self.config.metric_file_name, artifact_path="metrics")

            # Prepare example for MLflow signature
            input_example = test_x.iloc[:5]

            # Register the model
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    registered_model_name="ElasticnetModel",
                    input_example=input_example,
                )
            else:
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    input_example=input_example,
                )


In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-18 15:04:13,519: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-18 15:04:13,522: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-18 15:04:13,524: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-18 15:04:13,526: INFO: common: created directory at: artifacts]
[2025-09-18 15:04:13,526: INFO: common: created directory at: artifacts/model_evaluation]
[2025-09-18 15:04:14,039: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/09/18 15:04:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.


🏃 View run unique-chimp-734 at: https://dagshub.com/AshwinBalaji007/End-to-End-MLOps-Pipeline.mlflow/#/experiments/0/runs/a07e7b0deb604aa9b39fd0dc250ed132
🧪 View experiment at: https://dagshub.com/AshwinBalaji007/End-to-End-MLOps-Pipeline.mlflow/#/experiments/0
